In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("whitefish_NLP").getOrCreate()

In [ ]:
path="/content/drive/My Drive/Colab Notebooks/Resort_reviews_data_DRive"

In [ ]:
df = spark.read.csv("/content/drive/My Drive/Colab Notebooks/Resort_reviews_data_DRive/whitefish_y.csv", header=True, inferSchema=True)

In [ ]:
df.show(5)

+--------------------+--------------------+
|               class|                text|
+--------------------+--------------------+
|            positive|Perfect way to sp...|
|We love the alpin...| but would defini...|
|            positive|Have nothing but ...|
|And the resort wa...| with easy lift a...|
|I do have a bit t...| I had never been...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
# Import functions
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [ ]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature
data_df = df.withColumn('length', length(df['text']))
data_df.show()

+--------------------+--------------------+------+
|               class|                text|length|
+--------------------+--------------------+------+
|            positive|Perfect way to sp...|   295|
|We love the alpin...| but would defini...|    33|
|            positive|Have nothing but ...|   243|
|And the resort wa...| with easy lift a...|    22|
|I do have a bit t...| I had never been...|    53|
|It was a bit of a...| but you could no...|    26|
|Second surprise. ...| well that wasn't...|    86|
|Surprise number t...| that's 12.6 mile...|    41|
|I had a pretty gr...| especially befor...|    74|
|            positive|After snowboardin...|   581|
|I was able to boa...| amazing location...|   119|
|For the price and...| it's absolutely ...|    25|
|            negative|Beautiful views a...|   309|
|A few concerns wi...| on Sunday when s...|    64|
|They are also big...| even though the ...|    96|
|    It was good trip| but had better e...|    38|
|            negative|Staff tra

In [ ]:
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [ ]:
# Create and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [ ]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data_df)
cleaned = cleaner.transform(data_df)

Py4JJavaError: ignored